<a href="https://colab.research.google.com/github/hwaneest/pseudo_lab_tutorial_book/blob/main/Ch3_True_False_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. True/False Question

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pseudo-Lab/Tutorial-Book/blob/master/book/chapters/NLP/Ch3-True-False-Question.ipynb)

이전 장에서는 실습에 사용할 데이터셋을 다운로드하고 시각화하며 전처리를 진행해보았습니다. 이번 장에서는 해당 데이터셋을 이용하여 True/False 문제를 생성하는 모델을 실습해보도록 하겠습니다.
    
(수정중)    
3.1절에서는 모델링에 사용할 GPT2와 BERT에 대해 간단하게 설명하고, 3.2절에서는 데이터를 불러와서 다시 한 번 살펴봅니다. 그리고 3.3절에서는 ㅇㅇㅇ를 이용하여 해당 text 데이터를 요약(Summarize)하는 작업을 진행합니다. 마지막 3.4절에서는 Text 문장들을 필터링하고 GPT2와 BERT를 이용하여 True/False 문제를 생성해보도록 하겠습니다.


## 3.1 데이터셋 다운로드

2절에서 나온 코드를 활용하여 데이터셋을 불러오도록 하겠습니다. 데이터셋은 올바른 표현으로 수정이 된 에세이 텍스트 데이터입니다. 이 데이터는 True/False 문제를 만들 지문으로 사용됩니다. 데이터를 읽어 오기 위해서 pickle 패키지를 이용합니다.

In [22]:
import pickle

In [7]:
!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data NLP-QG
file_name = "CoNLL+BEA_corrected_essays.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

Cloning into 'Tutorial-Book-Utils'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 9), reused 18 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.
CoNLL+BEA_corrected_essays.pkl is done!


In [20]:
print('** What is the type of data? :', type(data))
print('** How many data are there? :', len(data))
print('** How long is the first sample data? :', len(data[0]))

** What is the type of data? : <class 'list'>
** How many data are there? : 170
** How long is the first sample data? : 2587


데이터를 간단하게 살펴보면 위와 같습니다. 읽어온 데이터는 'list'의 형태로 data 객체에 저장되었고, 이 리스트는 총 170개의 데이터를 요소로 가지고 있음을 알 수 있습니다. 리스트에서 0번째 요소인 첫번째 데이터의 길이는 2,587글자임을 알 수 있습니다.

첫번째 데이터를 출력해보면 아래와 같습니다. data 객체가 list 데이터 타입이므로 0번째 요소가 곧 첫번째 데이터를 의미합니다.

In [21]:
data[0]

'Keeping the Secret of Genetic Testing What is genetic risk? Genetic risk refers  to your chance of inheriting a disorder or disease. People get certain diseases because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However, for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagnosed with certain genetic diseases, are we suppose to disclose this result to our relatives? My answer is no. On one hand, we do not want this potential danger havingfrightening effects in our families\' later lives. W

어3장에서 사용할 패키지들을 import해보겠습니다. 각 환경에 따라 패키지가 없거나 버전 차이가 있을 수 있습니다. 아래 코드를 통해 패키지를 설치하거나 버전을 맞추어 사용하시기 바랍니다.

(수정중) 패키지 설명 

In [ ]:
### COLAB 내장 패키지들 ###
# requirements
# !pip install transformers==4.1.1
# !pip install spacy==2.3.1
# !pip install sentencepiece==0.1.94
# !pip install torch
# !python -m spacy download en_core_web_sm

In [ ]:
# |!pip install tensorflow==1.14.0   
# !pip install torch==1.4.0  
# !pip install sentence-transformers==0.2.5.1  
# !pip install transformers==2.6.0  
# !pip install benepar==0.1.2  
# !pip install summa  
# !pip install nltk==3.4.5  
# !pip install spacy==2.1.0  
# !python3 -m spacy download en  
# !pip install scipy  

In [ ]:
import requests
import json
from summa.summarizer import summarize
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import re
from random import shuffle
import spacy
import warnings
warnings.filterwarnings(action='ignore')
nlp = spacy.load('en')

#this package is required for the summa summarizer
nltk.download('punkt')
benepar.download('benepar_en2')
benepar_parser = benepar.Parser("benepar_en2")

## 3.2 Text Summarization

#### Summarize the loaded content

In [ ]:
text = 'There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”'

In [ ]:
from string import punctuation

def preprocess(sentences):
    output = []
    for sent in sentences:
        single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
        double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
        question_present = "?" in sent
        if single_quotes_present or double_quotes_present or question_present :
            continue
        else:
            output.append(sent.strip(punctuation))
    return output
        
        
def get_candidate_sents(resolved_text, ratio=0.3):
    candidate_sents = summarize(resolved_text, ratio=ratio)
    candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
    candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
    # Remove very short sentences less than 30 characters and long sentences greater than 150 characters
    filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)>30 and len(sent)<150]
    return filtered_list_short_sentences

cand_sents = get_candidate_sents(text)
filter_quotes_and_questions = preprocess(cand_sents)
for each_sentence in filter_quotes_and_questions:
    print (each_sentence)
    print ("\n")

As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust


Divergent plate boundaries also occur in the continental crust


Volcanoes form at these boundaries, but less often than in ocean crust


Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone




### GPT2
GPT2는 ...  설명 제작중...
  
  

### BERT
BERT는...  설명 제작중...

## 3.3 True/False 문제 생성

### GPT2 & BERT ...

#### split sentences at appropriate place¶

In [ ]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final
    

def get_termination_portion(main_string,sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
                     
    return None
    
def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(last_NP),get_flattened(last_VP)
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in filter_quotes_and_questions:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            phrases.append(nounphrase_string)

        longest_phrase =  sorted(phrases, key=len,reverse= True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase
    return sentence_completion_dict



sent_completion_dict = get_sentence_completions(filter_quotes_and_questions)

print (sent_completion_dict)

{'As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust': ['As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in'], 'Divergent plate boundaries also occur in the continental crust': ['Divergent plate boundaries also occur in', 'Divergent plate boundaries also'], 'Volcanoes form at these boundaries, but less often than in ocean crust': ['Volcanoes form at these boundaries, but less often than in'], 'Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone': ['Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at']}


# Load OpenAI GPT2 and Sentence BERT

In [ ]:
# https://huggingface.co/transformers/main_classes/model.html?highlight=no_repeat_ngram_size

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)

from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:18<00:00, 22.0MB/s]


# Filter sentences and generate false sentences.

In [ ]:
from nltk import tokenize
import scipy
import torch
torch.manual_seed(2020)


def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]
    

def generate_sentences(partial_sentence,full_sentence):
    input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
    maximum_length = len(partial_sentence.split())+80

    # Actiavte top_k sampling and top_p sampling with only from 90% most likely words
    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, # 0.85 
        top_k=50,   #0.30
        repetition_penalty  = 10.0,
        num_return_sequences=10
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0])
        
    top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
    
    return top_3_sentences

index = 1
choice_list = ["a)","b)","c)","d)","e)","f)"]
for key_sentence in sent_completion_dict:
    partial_sentences = sent_completion_dict[key_sentence]
    false_sentences =[]
    print_string = "**%s) True Sentence (from the story) :**"%(str(index))
    printmd(print_string)
    print ("  ",key_sentence)
    for partial_sent in partial_sentences:
        false_sents = generate_sentences(partial_sent,key_sentence)
        false_sentences.extend(false_sents)
    printmd("  **False Sentences (GPT-2 Generated)**")
    for ind,false_sent in enumerate(false_sentences):
        print_string_choices = "**%s** %s"%(choice_list[ind],false_sent)
        printmd(print_string_choices)
    index = index+1
    
    print ("\n\n")
        

**1) True Sentence (from the story) :**

   As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust


  **False Sentences (GPT-2 Generated)**

**a)** As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust that provide access to oxygen-rich water.

**b)** As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the seafloor that are more sensitive to wind velocity and pressure than most continental surfaces.

...

   Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone


  **False Sentences (GPT-2 Generated)**

**a)** Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a rate of 1.1, 3–4 km/h (5).

**b)** Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at about 25% of its original location.

**c)** Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a rate of about 100 million km/year.

#### Reference
 - https://medium.com/swlh/practical-ai-automatically-generate-true-or-false-questions-from-any-content-with-openai-gpt2-9081ffe4d4c9